# Match - missing biogenic and fossil carbon

SimaPro and Ecoinvent use different terms for similar or identical concepts. From section 7.3.3 of the [3.9 LCIA implementation report](https://ecoinvent.org/wp-content/uploads/2022/12/LCIA_implementation_3.9.pdf):

```
7.3.3.1 Carbon dioxide
Uptake of CO2 by plants is accounted for in forestry and agriculture datasets by the elementary exchange “Carbon dioxide, in air”. This same carbon eventually goes back to the atmosphere under the form of methane, CO, and CO2. Every dataset releasing these substances differentiates between the fossil and non-fossil origin of the carbon. The net null effect of capturing a carbon atom and releasing it later is modelled by attributing null CFs to resource elementary exchanges “Carbon dioxide, in air” and the emission elementary exchanges “Carbon dioxide, non-fossil” (Table 7).

...

7.3.3.3 Emissions from land use change (from soil or biomass stock)
Agriculture, forestry, land transformation and hydropower datasets also report emissions of carbon through the elementary exchanges “Carbon dioxide, from soil or biomass stock”, “Carbon monoxide, from soil or biomass stock” and “Methane, from soil or biomass stock”. These emissions are treated as fossil emissions (Table 7). Their CFs are therefore the same as their fossil counterpart, as they came from the atmosphere to the stock much earlier than the scope of any LCA, like fossil carbon. If there is a net carbon uptake in these datasets, this is reported through the elementary exchange “Carbon dioxide, to soil or biomass stock”, which gets -1 as a CF (Table 7).
```

**Table 7**: CFs for fossil and non-fossil carbon emissions in the implementation of IPCC 2021
| Substance name in ecoinvent | GWP100 | Context |
| --- | --- | --- |
| Carbon dioxide, in air | 0 | Natural Resource |
| Carbon dioxide, non-fossil | 0 | Emission to air |
| Carbon dioxide, fossil | 1 | Emission to air |
| Carbon dioxide, from soil or biomass stock | 1 | Emission to air |
| Carbon dioxide, to soil or biomass stock | -1 | Emission to soil |

Similar analysis is used for methane.

SimaPro has more flavors of Carbon Dioxide:

* Carbon dioxide
* Carbon dioxide, biogenic
* Carbon dioxide, fossil
* Carbon dioxide, land transformation
* Carbon dioxide, peat oxidation

As such, we can't do perfect two-way conversion, but will use the following SimaPro to ecoinvent conversion table *for air emissions*:

| SimaPro Flowable | ecoinvent Flowable | Comment |
| --- | --- | --- |
| Carbon dioxide | Carbon dioxide, fossil | Conservative assumption |
| Carbon dioxide, biogenic | Carbon dioxide, non-fossil | |
| Carbon dioxide, fossil | Carbon dioxide, fossil |
| Carbon dioxide, land transformation | Carbon dioxide, from soil or biomass stock | Same concept |
| Carbon dioxide, peat oxidation | Carbon dioxide, from soil or biomass stock |

We will apply the same for methane and carbon monixide.

In [1]:
import pandas as pd
from pathlib import Path
from notebook_utils import finish_notebook, expand_simapro_context, add_ecoinvent_context_column

Get paths of input and output directories

In [2]:
input_data_dir = (Path.cwd().parent / "Mapping" / "Input" / "Flowlists").resolve()
existing_matches_dir = (Path.cwd().parent / "Mapping" / "Output" / "Mapped_files").resolve()

Read input dataframes

In [3]:
sp = pd.read_csv(input_data_dir / 'SimaProv9.4.csv')

In [4]:
ei = pd.read_csv(input_data_dir / 'ecoinventEFv3.7.csv')

## With great `merge` comes great responsibility

We need to expand each row in `sp` with the context `Airborne emissions` to all possible SimaPro LCI contexts.

In [5]:
sp = expand_simapro_context(sp)

We then add a column `EcoinventContext` which maps context values from SimaPro to ecoinvent.

In [6]:
sp = add_ecoinvent_context_column(sp, "EcoinventContext")

In [7]:
sp[sp.Flowable == "Carbon dioxide"]

,Flowable,CAS No,Formula,Synonyms,Unit,Class,ParentContext,Flow UUID,Description,Context,EcoinventContext
13573,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/(unspecified),air/unspecified
13574,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/indoor,air/indoor
13575,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/high. pop.,air/urban air close to ground
13576,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/low. pop.,air/non-urban air or from high stacks
13577,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,"Emissions to air/low. pop., long-term","air/low population density, long-term"
13578,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/stratosphere + troposphere,air/lower stratosphere + upper troposphere
13579,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/stratosphere,air/lower stratosphere + upper troposphere


Finally, we map the specific name pairs and merge using the added `EcoinventContext` column.

In [8]:
ecoinvent_mapper = pd.DataFrame([
    ("Carbon dioxide", "Carbon dioxide, fossil"),
    ("Carbon dioxide, biogenic", "Carbon dioxide, non-fossil"),
    ("Carbon dioxide, fossil", "Carbon dioxide, fossil"),
    ("Carbon dioxide, land transformation", "Carbon dioxide, from soil or biomass stock"),
    ("Carbon dioxide, peat oxidation", "Carbon dioxide, from soil or biomass stock"),
    ("Methane", "Methane, fossil"),
    ("Methane, biogenic", "Methane, non-fossil"),
    ("Methane, fossil", "Methane, fossil"),
    ("Methane, land transformation", "Methane, from soil or biomass stock"),
    ("Methane, peat oxidation", "Methane, from soil or biomass stock"),
    ("Carbon monoxide", "Carbon monoxide, fossil"),
    ("Carbon monoxide, biogenic", "Carbon monoxide, non-fossil"),
    ("Carbon monoxide, fossil", "Carbon monoxide, fossil"),
    ("Carbon monoxide, land transformation", "Carbon monoxide, from soil or biomass stock"),
], columns=["Flowable", "EcoinventFlowable"])

In [9]:
sp = sp.merge(ecoinvent_mapper, how="outer", on="Flowable")

In [10]:
df = sp.merge(
    ei, 
    how='inner', 
    left_on=["EcoinventContext", "EcoinventFlowable"], 
    right_on=["Context", "Flowable"], 
)

In [11]:
df[df.Flowable_x == "Carbon dioxide"]

,Flowable_x,CAS No,Formula_x,Synonyms_x,Unit_x,Class_x,ParentContext,Flow UUID,Description,Context_x,...,Synonyms_y,Unit_y,Class_y,ExternalReference,Preferred,Context_y,FlowUUID,AltUnit,Unnamed: 11,Second CAS
0,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/(unspecified),...,NaN,kg,chemical,NaN,NaN,air/unspecified,349b29d1-3e58-4c66-98b9-9d1a076efd2e,NaN,NaN,NaN
2,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/high. pop.,...,NaN,kg,chemical,NaN,NaN,air/urban air close to ground,f9749677-9c9f-4678-ab55-c607dfdc2cb9,NaN,NaN,NaN
4,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/low. pop.,...,NaN,kg,chemical,NaN,NaN,air/non-urban air or from high stacks,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,NaN,NaN,NaN
6,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,"Emissions to air/low. pop., long-term",...,NaN,kg,chemical,NaN,NaN,"air/low population density, long-term",e259263c-d1f1-449f-bb9b-73c6d0a32a00,NaN,NaN,NaN
8,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/stratosphere + troposphere,...,NaN,kg,chemical,NaN,NaN,air/lower stratosphere + upper troposphere,16eeda8a-1ea2-408e-ab37-2648495058dd,NaN,NaN,NaN
9,Carbon dioxide,000124-38-9,NaN,Carbon dioxide,kg,Airborne emissions,Airborne emissions,DBE27C73-9DF4-4F21-B2EB-A0CE1FE87A4B,Formula: CO2,Emissions to air/stratosphere,...,NaN,kg,chemical,NaN,NaN,air/lower stratosphere + upper troposphere,16eeda8a-1ea2-408e-ab37-2648495058dd,NaN,NaN,NaN


In [12]:
finish_notebook(
    df=df,
    author="Chris Mutel",
    notebook_name="Match - missing biogenic and fossil carbon",
    filename="carbon-missing",
)